In [1]:
def new_board():
    # startujemy nowa plansze
    return ((0,0,0),(0,0,0),(0,0,0))

In [2]:
def apply_move(board_state, move, side):
    # rozdzielamy współrzedne ruchu
    move_x, move_y = move
    # zapisujemy stan gry jako lista list
    state_list = list(list(s) for s in board_state)
    # zapisujemy strone gry jako 1 albo -1 w danych współrzednych
    state_list[move_x][move_y] = side
    # zwracamy spowrotem niezmienialny nawias nawiasow 
    return tuple(tuple(s) for s in state_list)

In [3]:
import itertools
def available_moves(board_state):
    # iterujemy po wszystkich nawaiasach nawiasow
    for x,y in itertools.product(range(3), range(3)):
        # zwracamy kratke gdzie jest 0, czyli prawidlowe ruchy
        if board_state[x][y] == 0:
            yield (x,y)

In [4]:
def has_3_in_a_line(line):
    return all(x==-1 for x in line) | all(x==1 for x in line)

In [5]:
# sprawdzamy czy nie ma wygranej na planszy
def has_winner(board_state):
    # sprawdzanie rzedów
    for x in range(3):
        # bierzemyh linie
        if has_3_in_a_line(board_state[x]):
            return board_state[x][0]
    # sprawdzanie kolun
    for y in range(3):
        # bierzemy i pozycje z kazdej lini
        if has_3_in_a_line([i[y] for i in board_state]):
            return board_state[0][y]
    # sprawdzanie przekotnych
    # sprawdzanie przekatnej \
    if has_3_in_a_line([board_state[i][i] for i in range(3)]):
        return board_state[0][0]
    if has_3_in_a_line([board_state[2-i][i] for i in range(3)]):
        return board_state[2][0]
    return 0 # remis

In [6]:
# glówna funkcja gry
def play_game(plus_player_func, minus_player_func):
    board_state = new_board()
    # gracz 1 zaczyna, i towżymy zmian ktora bd ustalala ture
    player_turn = 1
    while True:
        # pobieramy liste wszystkich mozliwych róchow 
        _available_moves = list(available_moves(board_state))
        if len(_available_moves) == 0:
            print("brak dostepnych ruchów, remis")
            return 0
        # ustalamy czyja jest tura
        if player_turn > 0:
            move = plus_player_func(board_state, 1)
        else:
            move = minus_player_func(board_state, -1)
        # jesli agent wykona niedozwolony ruch, to przeciwnik wygrywa
        if move not in _available_moves:
            print("niedozwolony ruch")
            return -player_turn
        # wykonujemy ruch
        board_state = apply_move(board_state, move, player_turn)
        print(board_state)
        # sprawdzamy czy ktos wygral 
        winner = has_winner(board_state)
        if winner != 0:
            print("winer is : %s" % player_turn)
            return winner
        # i oddajemy na następna ture 
        player_turn = -player_turn

In [7]:
def random_player(board_state, side):
    import random
    moves = list(available_moves(board_state))
    return random.choice(moves)

In [8]:
play_game(random_player, random_player)

((1, 0, 0), (0, 0, 0), (0, 0, 0))
((1, -1, 0), (0, 0, 0), (0, 0, 0))
((1, -1, 1), (0, 0, 0), (0, 0, 0))
((1, -1, 1), (0, 0, 0), (0, 0, -1))
((1, -1, 1), (0, 1, 0), (0, 0, -1))
((1, -1, 1), (0, 1, 0), (0, -1, -1))
((1, -1, 1), (1, 1, 0), (0, -1, -1))
((1, -1, 1), (1, 1, 0), (-1, -1, -1))
winer is : -1


-1

In [10]:
# teraz bot min-max
# funkcja liczaca punkty, daje na 1 punk za 2 symbole w lini i w przypadku gdy trzecie pole jest puste
def score_line(line):
    minus_count = line.count(-1)
    plus_count = line.count(1)
    if plus_count == 2 and minus_count ==0:
        return 1
    elif minus_count == 2 and plus_count ==0:
        return -1
    return 0

In [13]:
# funkcja pobierajaca wszystkie mozliwe linie i je sumuje
def evaluate(board_state):
    score  = 0
    for x in range(3):
        score += score_line(board_state[x])
    for y in range(3):
        score += score_line([i[y] for i in board_state])
    # sumujemz przekątne
    score += score_line(board_state[i][i] for i in range(3))
    score += score_line(board_state[2-i][i] for i in range(3))
    
    return score

In [ ]:
# algorytm min-max
def min_max(board_state, side, max_depth):
    # max_depth to argument rekurencyjny mowi ile wywołan zrobimy czyli max glebokosc
    # max_depth za kazdym wywolaniem bd zmniejszane o 1
    best_score = 0
    best_score_move = 0
    moves = list(available_moves(board_state))
    if not moves:
        return 0
    for move in moves:
        new_board_state = apply_move(board_state, move, side)
    winner = has_winner(new_board_state)
    if winner !=0:
        return winner*1000, move
    else:
        if max_depth<=1:
            score = evaluate(new_board_state)
        else:
            score, _ = min_max(new_board_state, -side,max_depth-1)
    if side>0:
        # max
        if best_score is None or score>best_score:
            best_score = score
            best_score_move =move
        # min
        if best_score is None or score<best_score:
            best_score = score
            best_score_move =move
    return best_score, best_score_move

In [ ]:
def min_max_player(board_state, side):
    return min_max(board_state, side, 5)[1]